<a href="https://colab.research.google.com/github/merucode/DL/blob/01-colab-study_must_have_pytorch/03-03_basic_classification_%20handwriting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## STEP 0. Version check and Install Dependency

Step 0-1. Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.version}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

Step 0-2. Install Dependency

## STEP 1. Check Data

Step 1-1. Load data

In [ ]:
import matplotlib.pyplot as plt

from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor

# ❶ 학습용 데이터와 평가용 데이터 분리
training_data = MNIST(root="./", train=True, download=True, transform=ToTensor())
test_data = MNIST(root="./", train=False, download=True, transform=ToTensor())

print(len(training_data)) # 학습에 사용할 데이터 개수
print(len(test_data))     # 평가에 사용할 데이터 개수

Step 1-2. Check data type

In [ ]:
for i in range(9): # 샘플 이미지를 9개 출력
   plt.subplot(3, 3, i+1)
   plt.imshow(training_data.data[i])
plt.show()

## STEP 2. Dataset

In [ ]:
from torch.utils.data.dataloader import DataLoader

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)

# ❶평가용은 데이터를 섞을 필요가 없음
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

## STEP 3. Module

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
   nn.Linear(784, 64),
   nn.ReLU(),
   nn.Linear(64, 64),
   nn.ReLU(),
   nn.Linear(64, 10)
)

## STEP 4. Learning

Step 4-1. Setting

In [ ]:
from torch.optim.adam import Adam

device = "cuda" if torch.cuda.is_available() else "cpu" # ❶ 학습에 사용할 프로세서를 지정
model.to(device) # 모델의 파라미터를 GPU로 보냄

lr = 1e-3
optim = Adam(model.parameters(), lr=lr)

Step 4-2. Learning

In [ ]:
for epoch in range(20):
   for data, label in train_loader:
       optim.zero_grad()  # 가중치 초기화

       data = torch.reshape(data, (-1, 784)).to(device)       # ❷ 입력 데이터를 모델의 입력에 맞게 모양을 변환
       # -1은 갯수 상관 않음. 픽셀이미지를 일렬로 (32, 28, 28 ) → (32, 784)
       preds = model(data) # prediction

       loss = nn.CrossEntropyLoss()(preds, label.to(device))  # ❸ 손실 계산
       loss.backward()  # backward
       optim.step()     # update w

   print(f"epoch{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "MNIST.pt") # ➍ 모델을 MNIST.pt라는 이름으로 저장

## STEP 5. Evaluation

In [ ]:
# ❶ 모델 가중치 불러오기
model.load_state_dict(torch.load("MNIST.pt", map_location=device))

num_corr = 0 # 분류에 성공한 전체 개수

with torch.no_grad(): # ❷ 기울기를 계산하지 않음
   for data, label in test_loader:
       data = torch.reshape(data, (-1, 784)).to(device)

       output = model(data.to(device))
       preds = output.data.max(1)[1] # ❸ 모델의 예측값 계산
       # ❹ 올바르게 분류한 개수
       corr = preds.eq(label.to(device).data).sum().item()
       num_corr += corr

   print(f"Accuracy:{num_corr/len(test_data)}") # 분류 정확도를 출력합니다.